In [4]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%

## Data Preparation

In [5]:
from sklearn.decomposition import PCA
from tensorflow.keras import layers, callbacks
from sklearn.model_selection import RandomizedSearchCV, train_test_split;
import numpy;
import pandas;
from sklearn.pipeline import Pipeline
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier;
import sys;
from keras.utils import to_categorical;
REMOTE_MODULE_PATH = '/home/macbookretina/automatic-music-genre-classification/feature_extraction_deep_learning'
# LOCAL_MODULE_PATH = '/Users/macbookretina/Desktop/automatic-music-genre-classification/feature_extraction_deep_learning'
sys.path.insert(1, REMOTE_MODULE_PATH)
from custom_module.utilities import *

In [6]:
features = pandas.read_csv(MOUNTED_DATASET_PATH + '/data/cleaned_features.csv', index_col=0)
features

,genre_label,lpc_1,lpc_2,lpc_3,lpc_4,mean_beats,mean_beats_timestamp,mean_db4_cA4,mean_db4_cD1,mean_db4_cD2,...,var_spec_centroid,var_spec_contrast_1,var_spec_contrast_2,var_spec_contrast_3,var_spec_contrast_4,var_spec_contrast_5,var_spec_contrast_6,var_spec_contrast_7,var_spec_rolloff,var_zcr
0,0,1.0,-1.178271,0.647402,-0.352129,649.120690,15.072553,-0.070889,2.706283e-09,-1.245192e-04,...,4.185230e+05,17.861575,21.121083,25.873245,13.038216,6.962883,7.988678,14.555389,1.564158e+06,0.002891
1,0,1.0,-1.086489,0.542333,-0.396561,617.465116,14.337512,-0.000740,8.956381e-06,-9.212386e-05,...,9.392262e+05,29.258739,24.407069,23.032866,18.185633,12.981837,12.461155,107.433159,2.076888e+06,0.005135
2,0,1.0,-1.365355,0.560194,-0.172092,631.428571,14.661743,-0.011143,-1.990486e-09,-7.116798e-06,...,6.335591e+05,14.853086,20.595163,23.554291,21.775710,11.043003,11.478725,17.035474,3.405313e+06,0.002481
3,0,1.0,-1.240807,0.505005,-0.190461,619.227273,14.378429,-0.002008,3.508618e-05,-2.028877e-05,...,4.953543e+05,25.631542,17.320487,26.199990,13.617381,7.566584,8.344646,142.327995,1.811020e+06,0.002283
4,0,1.0,-0.962111,0.201135,-0.206167,631.382979,14.660684,-0.001342,3.828920e-05,-4.344582e-05,...,1.073302e+06,35.138048,34.537389,32.614328,30.967176,19.603025,10.813280,102.154238,4.354818e+06,0.006124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294,0,1.0,-1.878301,1.492194,-0.602686,1246.841270,14.475799,0.000255,-3.682523e-07,1.021450e-05,...,6.717626e+05,25.312044,16.848874,13.235958,17.875383,10.292653,7.975512,70.855880,3.494444e+06,0.001376
3295,0,1.0,-0.757695,0.322183,-0.531011,1269.770833,14.742011,0.000069,-3.853609e-07,3.565250e-05,...,9.118337e+06,36.373645,31.511738,26.118394,30.467805,48.009289,32.720426,98.557969,2.956056e+07,0.007543
3296,2,1.0,-1.862517,1.301521,-0.433298,1254.981818,14.570310,0.000784,-1.326012e-06,6.062675e-07,...,9.370093e+05,47.880472,15.887094,19.648933,20.805391,14.361217,9.615398,48.662283,6.126479e+06,0.000358
3297,2,1.0,-1.882439,1.470879,-0.573606,1272.877551,14.778080,-0.000206,-3.779051e-07,2.349206e-05,...,1.390328e+06,32.331749,18.345547,18.593669,21.698056,14.748304,8.958216,19.165002,5.990872e+06,0.001409


In [7]:
y = features.loc[:,'genre_label']
X = features.loc[:, features.columns.difference(['genre_label'])]
X

,lpc_1,lpc_2,lpc_3,lpc_4,mean_beats,mean_beats_timestamp,mean_db4_cA4,mean_db4_cD1,mean_db4_cD2,mean_db4_cD3,...,var_spec_centroid,var_spec_contrast_1,var_spec_contrast_2,var_spec_contrast_3,var_spec_contrast_4,var_spec_contrast_5,var_spec_contrast_6,var_spec_contrast_7,var_spec_rolloff,var_zcr
0,1.0,-1.178271,0.647402,-0.352129,649.120690,15.072553,-0.070889,2.706283e-09,-1.245192e-04,-0.000069,...,4.185230e+05,17.861575,21.121083,25.873245,13.038216,6.962883,7.988678,14.555389,1.564158e+06,0.002891
1,1.0,-1.086489,0.542333,-0.396561,617.465116,14.337512,-0.000740,8.956381e-06,-9.212386e-05,0.000984,...,9.392262e+05,29.258739,24.407069,23.032866,18.185633,12.981837,12.461155,107.433159,2.076888e+06,0.005135
2,1.0,-1.365355,0.560194,-0.172092,631.428571,14.661743,-0.011143,-1.990486e-09,-7.116798e-06,0.000019,...,6.335591e+05,14.853086,20.595163,23.554291,21.775710,11.043003,11.478725,17.035474,3.405313e+06,0.002481
3,1.0,-1.240807,0.505005,-0.190461,619.227273,14.378429,-0.002008,3.508618e-05,-2.028877e-05,-0.001993,...,4.953543e+05,25.631542,17.320487,26.199990,13.617381,7.566584,8.344646,142.327995,1.811020e+06,0.002283
4,1.0,-0.962111,0.201135,-0.206167,631.382979,14.660684,-0.001342,3.828920e-05,-4.344582e-05,0.000076,...,1.073302e+06,35.138048,34.537389,32.614328,30.967176,19.603025,10.813280,102.154238,4.354818e+06,0.006124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294,1.0,-1.878301,1.492194,-0.602686,1246.841270,14.475799,0.000255,-3.682523e-07,1.021450e-05,-0.000065,...,6.717626e+05,25.312044,16.848874,13.235958,17.875383,10.292653,7.975512,70.855880,3.494444e+06,0.001376
3295,1.0,-0.757695,0.322183,-0.531011,1269.770833,14.742011,0.000069,-3.853609e-07,3.565250e-05,-0.000009,...,9.118337e+06,36.373645,31.511738,26.118394,30.467805,48.009289,32.720426,98.557969,2.956056e+07,0.007543
3296,1.0,-1.862517,1.301521,-0.433298,1254.981818,14.570310,0.000784,-1.326012e-06,6.062675e-07,-0.000024,...,9.370093e+05,47.880472,15.887094,19.648933,20.805391,14.361217,9.615398,48.662283,6.126479e+06,0.000358
3297,1.0,-1.882439,1.470879,-0.573606,1272.877551,14.778080,-0.000206,-3.779051e-07,2.349206e-05,-0.000132,...,1.390328e+06,32.331749,18.345547,18.593669,21.698056,14.748304,8.958216,19.165002,5.990872e+06,0.001409


In [8]:
y.value_counts(normalize=True)

2    0.333434
0    0.333434
1    0.333131
Name: genre_label, dtype: float64

In [9]:
predictors_all_label = X.columns.values
predictors_with_outliers_label = ['mean_beats', 'mean_spec_bw', 'mean_spec_centroid', 'mean_spec_rolloff',
                          'median_beats','median_spec_bw', 'median_spec_centroid', 
                          'median_spec_rolloff', 'std_beats', 'std_mfcc_1', 'var_spec_centroid',
                          'var_spec_rolloff'
                         ]
predictors_without_outliers_label = X.loc[
    :, X.columns.difference(predictors_with_outliers_label)].columns.values

In [10]:
preprocess_sp = standardization_pipeline(
    predictors_all_label,
    predictors_with_outliers_label,
    predictors_without_outliers_label)

preprocess_np = normalization_pipeline(
    predictors_all_label)

### Split & Transform Data

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True)
X_train.shape

(2639, 217)

In [12]:
pipeline_estimator_sp = standardization_pipeline(
    predictors_all_label,
    predictors_with_outliers_label,
    predictors_without_outliers_label
).fit(X_train)
# X_train_sp = pipeline_estimator_sp.transform(X_train)
X_test_sp = pipeline_estimator_sp.transform(X_test)
X_test_sp.shape

(660, 217, 1)

In [13]:
pipeline_estimator_np = normalization_pipeline(
    predictors_all_label).fit(X_train)
# X_train_np = pipeline_estimator_np.transform(X_train)
X_test_np = pipeline_estimator_np.transform(X_test)
X_test_np.shape

(660, 217, 1)

In [14]:
_, ncols, _ = X_test_np.shape

In [15]:
# one hot encoding our labels
y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)

In [16]:
y_train.shape, y_test.shape

((2639, 3), (660, 3))

## Predictive Modelling

In [17]:
early_stopping_cb = callbacks.EarlyStopping(patience=10,
                                        restore_best_weights=True)

In [18]:
param_dist = {
    'clf__n_hidden': [0,1,2,3,4,5],
    'clf__activation': ['relu', 'elu', 'selu'],
    'clf__optimizer': ['rmsprop','adam','adagrad'],
    'clf__kernel_initializer':['glorot_uniform','normal','uniform'],
    'clf__n_neurons': numpy.arange(1,100)
}

### FNN With Standardized Input

In [19]:
# wrap the function with keras wrapper
fnn_1 = KerasClassifier(build_fn=set_shape_create_model('fnn', ncols))

In [20]:
pipeline_sp = Pipeline([
    ('preprocess', preprocess_sp),
    ('clf', fnn_1)
])

In [21]:
# if you're not using a GPU, you can set n_jobs to something other than 1
rscv = RandomizedSearchCV(pipeline_sp, param_dist, cv=3, n_jobs=1)

In [22]:
# %%timeit
# fit model
search_1 = rscv.fit(X_train, y_train, clf__batch_size=32,
                    clf__validation_data=(X_test_sp, y_test),
                    clf__epochs=100, clf__callbacks=[early_stopping_cb])

Model: "fnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 217)               0         
_________________________________________________________________
dense (Dense)                (None, 88)                19184     
_________________________________________________________________
dense_1 (Dense)              (None, 88)                7832      
_________________________________________________________________
dense_2 (Dense)              (None, 88)                7832      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 267       
Total params: 35,115
Trainable params: 35,115
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
55/55 [==============================] - 0s 9ms/step - loss: 0.9816 - accuracy: 0.6032 - val_loss: 0.7484 - 

55/55 [==============================] - 0s 3ms/step - loss: 0.4068 - accuracy: 0.8426 - val_loss: 0.6805 - val_accuracy: 0.7364
Epoch 8/100
55/55 [==============================] - 0s 3ms/step - loss: 0.3619 - accuracy: 0.8631 - val_loss: 0.7380 - val_accuracy: 0.7030
Epoch 9/100
55/55 [==============================] - 0s 3ms/step - loss: 0.3284 - accuracy: 0.8773 - val_loss: 0.7055 - val_accuracy: 0.7273
Epoch 10/100
55/55 [==============================] - 0s 3ms/step - loss: 0.2979 - accuracy: 0.8926 - val_loss: 0.7685 - val_accuracy: 0.7121
Epoch 11/100
55/55 [==============================] - 0s 2ms/step - loss: 0.2599 - accuracy: 0.9034 - val_loss: 0.7566 - val_accuracy: 0.7379
Epoch 12/100
55/55 [==============================] - 0s 3ms/step - loss: 0.2427 - accuracy: 0.9080 - val_loss: 0.7965 - val_accuracy: 0.7258
Epoch 13/100
55/55 [==============================] - 0s 1ms/step - loss: 0.4986 - accuracy: 0.8023
Model: "fnn"
__________________________________________________

55/55 [==============================] - 0s 1ms/step - loss: 0.5059 - accuracy: 0.7936
Model: "fnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 217)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 26)                5668      
_________________________________________________________________
dense_25 (Dense)             (None, 26)                702       
_________________________________________________________________
dense_26 (Dense)             (None, 26)                702       
_________________________________________________________________
dense_27 (Dense)             (None, 26)                702       
_________________________________________________________________
dense_28 (Dense)             (None, 26)                702       
__________________________________________

Epoch 1/100
55/55 [==============================] - 0s 3ms/step - loss: 1.1266 - accuracy: 0.5429 - val_loss: 0.9039 - val_accuracy: 0.6106
Epoch 2/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8599 - accuracy: 0.6299 - val_loss: 0.8307 - val_accuracy: 0.6591
Epoch 3/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7943 - accuracy: 0.6617 - val_loss: 0.7952 - val_accuracy: 0.6652
Epoch 4/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7536 - accuracy: 0.6771 - val_loss: 0.7748 - val_accuracy: 0.6864
Epoch 5/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7239 - accuracy: 0.6924 - val_loss: 0.7481 - val_accuracy: 0.7000
Epoch 6/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7024 - accuracy: 0.7032 - val_loss: 0.7510 - val_accuracy: 0.6955
Epoch 7/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6868 - accuracy: 0.7129 - val_loss: 0.7345 - val_accuracy: 0.7015
Epoch 8/100
5

55/55 [==============================] - 0s 4ms/step - loss: 0.8560 - accuracy: 0.6185 - val_loss: 0.7162 - val_accuracy: 0.6924
Epoch 2/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6870 - accuracy: 0.7095 - val_loss: 0.6919 - val_accuracy: 0.7076
Epoch 3/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6279 - accuracy: 0.7396 - val_loss: 0.6607 - val_accuracy: 0.7288
Epoch 4/100
55/55 [==============================] - 0s 2ms/step - loss: 0.5816 - accuracy: 0.7675 - val_loss: 0.6672 - val_accuracy: 0.7258
Epoch 5/100
55/55 [==============================] - 0s 2ms/step - loss: 0.5421 - accuracy: 0.7840 - val_loss: 0.6725 - val_accuracy: 0.7242
Epoch 6/100
55/55 [==============================] - 0s 2ms/step - loss: 0.5027 - accuracy: 0.7953 - val_loss: 0.7093 - val_accuracy: 0.7424
Epoch 7/100
55/55 [==============================] - 0s 2ms/step - loss: 0.4732 - accuracy: 0.8073 - val_loss: 0.7379 - val_accuracy: 0.7000
Epoch 8/100
55/55 [======

...

Epoch 74/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8058 - accuracy: 0.6566 - val_loss: 0.8209 - val_accuracy: 0.6515
Epoch 75/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8048 - accuracy: 0.6572 - val_loss: 0.8202 - val_accuracy: 0.6515
Epoch 76/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8039 - accuracy: 0.6595 - val_loss: 0.8196 - val_accuracy: 0.6515
Epoch 77/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8030 - accuracy: 0.6600 - val_loss: 0.8189 - val_accuracy: 0.6500
Epoch 78/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8021 - accuracy: 0.6600 - val_loss: 0.8183 - val_accuracy: 0.6500
Epoch 79/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8011 - accuracy: 0.6600 - val_loss: 0.8177 - val_accuracy: 0.6500
Epoch 80/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8003 - accuracy: 0.6617 - val_loss: 0.8171 - val_accuracy: 0.6500
Epoch 

55/55 [==============================] - 0s 2ms/step - loss: 0.8720 - accuracy: 0.6271 - val_loss: 0.8845 - val_accuracy: 0.6197
Epoch 27/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8689 - accuracy: 0.6271 - val_loss: 0.8822 - val_accuracy: 0.6197
Epoch 28/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8661 - accuracy: 0.6282 - val_loss: 0.8801 - val_accuracy: 0.6212
Epoch 29/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8634 - accuracy: 0.6299 - val_loss: 0.8780 - val_accuracy: 0.6273
Epoch 30/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8607 - accuracy: 0.6310 - val_loss: 0.8761 - val_accuracy: 0.6288
Epoch 31/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8582 - accuracy: 0.6310 - val_loss: 0.8742 - val_accuracy: 0.6303
Epoch 32/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8558 - accuracy: 0.6339 - val_loss: 0.8724 - val_accuracy: 0.6288
Epoch 33/100
55/55 

Epoch 84/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7869 - accuracy: 0.6652 - val_loss: 0.8227 - val_accuracy: 0.6636
Epoch 85/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7861 - accuracy: 0.6646 - val_loss: 0.8221 - val_accuracy: 0.6636
Epoch 86/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7853 - accuracy: 0.6652 - val_loss: 0.8215 - val_accuracy: 0.6636
Epoch 87/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7845 - accuracy: 0.6652 - val_loss: 0.8210 - val_accuracy: 0.6636
Epoch 88/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7837 - accuracy: 0.6652 - val_loss: 0.8204 - val_accuracy: 0.6636
Epoch 89/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7829 - accuracy: 0.6657 - val_loss: 0.8199 - val_accuracy: 0.6636
Epoch 90/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7822 - accuracy: 0.6657 - val_loss: 0.8193 - val_accuracy: 0.6636
Epoch 

55/55 [==============================] - 0s 2ms/step - loss: 0.8858 - accuracy: 0.6000 - val_loss: 0.8436 - val_accuracy: 0.6364
Epoch 37/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8835 - accuracy: 0.6006 - val_loss: 0.8419 - val_accuracy: 0.6333
Epoch 38/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8813 - accuracy: 0.6000 - val_loss: 0.8403 - val_accuracy: 0.6318
Epoch 39/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8793 - accuracy: 0.6006 - val_loss: 0.8387 - val_accuracy: 0.6333
Epoch 40/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8772 - accuracy: 0.6011 - val_loss: 0.8372 - val_accuracy: 0.6318
Epoch 41/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8752 - accuracy: 0.6011 - val_loss: 0.8358 - val_accuracy: 0.6348
Epoch 42/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8733 - accuracy: 0.6034 - val_loss: 0.8344 - val_accuracy: 0.6364
Epoch 43/100
55/55 

Epoch 94/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8110 - accuracy: 0.6392 - val_loss: 0.7908 - val_accuracy: 0.6667
Epoch 95/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8103 - accuracy: 0.6386 - val_loss: 0.7903 - val_accuracy: 0.6682
Epoch 96/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8095 - accuracy: 0.6403 - val_loss: 0.7898 - val_accuracy: 0.6682
Epoch 97/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8087 - accuracy: 0.6403 - val_loss: 0.7893 - val_accuracy: 0.6682
Epoch 98/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8079 - accuracy: 0.6403 - val_loss: 0.7888 - val_accuracy: 0.6682
Epoch 99/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8072 - accuracy: 0.6426 - val_loss: 0.7883 - val_accuracy: 0.6682
Epoch 100/100
55/55 [==============================] - 0s 1ms/step - loss: 0.8058 - accuracy: 0.6438
Model: "fnn"
__________________________________

In [23]:
search_1.best_score_

0.7146646611480019

In [24]:
search_1.best_params_

{'clf__activation': 'relu',
 'clf__kernel_initializer': 'uniform',
 'clf__n_hidden': 2,
 'clf__n_neurons': 59,
 'clf__optimizer': 'rmsprop'}

In [25]:
search_1.best_estimator_

Pipeline(memory=None,
     steps=[('preprocess', Pipeline(memory=None,
     steps=[('addcolumnnames', AddColumnNames(columns=array(['lpc_1', 'lpc_2', ..., 'var_spec_rolloff', 'var_zcr'], dtype=object))), ('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('predictors_with_outliers', Pipeline(memory=None,
     st..., ('clf', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fd1907720f0>)])

### FNN With Normalized Input

In [26]:
# wrap the function with keras wrapper
fnn_2 = KerasClassifier(set_shape_create_model('fnn_np', ncols))

In [27]:
pipeline_np = Pipeline([
    ('preprocess', preprocess_np),
    ('clf', fnn_2)
])

In [28]:
# if you're not using a GPU, you can set n_jobs to something other than 1
rscv = RandomizedSearchCV(pipeline_np, param_dist, cv=3, n_jobs=1)

In [29]:
# %%timeit
# fit model
search_2 = rscv.fit(X_train, y_train, clf__batch_size=32,
                    clf__validation_data=(X_test_np, y_test),
                    clf__epochs=100, clf__callbacks=[early_stopping_cb])

Model: "fnn_np"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_31 (Flatten)         (None, 217)               0         
_________________________________________________________________
dense_102 (Dense)            (None, 3)                 654       
Total params: 654
Trainable params: 654
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
55/55 [==============================] - 0s 3ms/step - loss: 1.0602 - accuracy: 0.4275 - val_loss: 1.0256 - val_accuracy: 0.4818
Epoch 2/100
55/55 [==============================] - 0s 8ms/step - loss: 0.9970 - accuracy: 0.5418 - val_loss: 0.9742 - val_accuracy: 0.5561
Epoch 3/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9506 - accuracy: 0.5753 - val_loss: 0.9591 - val_accuracy: 0.4939
Epoch 4/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9191 - accuracy: 0.60

Epoch 1/100
55/55 [==============================] - 0s 4ms/step - loss: 1.0615 - accuracy: 0.4554 - val_loss: 1.0352 - val_accuracy: 0.5136
Epoch 2/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9933 - accuracy: 0.5560 - val_loss: 0.9984 - val_accuracy: 0.5424
Epoch 3/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9463 - accuracy: 0.5918 - val_loss: 0.9413 - val_accuracy: 0.5864
Epoch 4/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9151 - accuracy: 0.6151 - val_loss: 0.9083 - val_accuracy: 0.5939
Epoch 5/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8888 - accuracy: 0.6128 - val_loss: 0.8916 - val_accuracy: 0.6091
Epoch 6/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8674 - accuracy: 0.6293 - val_loss: 0.8754 - val_accuracy: 0.6030
Epoch 7/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8583 - accuracy: 0.6202 - val_loss: 0.8560 - val_accuracy: 0.6318
Epoch 8/100
5

Epoch 59/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7003 - accuracy: 0.7106 - val_loss: 0.7106 - val_accuracy: 0.6970
Epoch 60/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7007 - accuracy: 0.7089 - val_loss: 0.7268 - val_accuracy: 0.6879
Epoch 61/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7046 - accuracy: 0.7032 - val_loss: 0.7074 - val_accuracy: 0.6909
Epoch 62/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7033 - accuracy: 0.7078 - val_loss: 0.7172 - val_accuracy: 0.6985
Epoch 63/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6962 - accuracy: 0.7163 - val_loss: 0.7081 - val_accuracy: 0.6955
Epoch 64/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6962 - accuracy: 0.7157 - val_loss: 0.7231 - val_accuracy: 0.6970
Epoch 65/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6983 - accuracy: 0.7169 - val_loss: 0.7121 - val_accuracy: 0.6955
Epoch 

55/55 [==============================] - 0s 2ms/step - loss: 0.8267 - accuracy: 0.6523 - val_loss: 0.8100 - val_accuracy: 0.6576
Epoch 12/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8169 - accuracy: 0.6534 - val_loss: 0.8055 - val_accuracy: 0.6530
Epoch 13/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8108 - accuracy: 0.6443 - val_loss: 0.7917 - val_accuracy: 0.6439
Epoch 14/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8052 - accuracy: 0.6614 - val_loss: 0.7864 - val_accuracy: 0.6636
Epoch 15/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8007 - accuracy: 0.6608 - val_loss: 0.7824 - val_accuracy: 0.6652
Epoch 16/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7950 - accuracy: 0.6636 - val_loss: 0.7818 - val_accuracy: 0.6697
Epoch 17/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7912 - accuracy: 0.6699 - val_loss: 0.7722 - val_accuracy: 0.6667
Epoch 18/100
55/55 

Epoch 69/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7089 - accuracy: 0.7063 - val_loss: 0.7057 - val_accuracy: 0.6955
Epoch 70/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7104 - accuracy: 0.6994 - val_loss: 0.7016 - val_accuracy: 0.7015
Epoch 71/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7096 - accuracy: 0.7051 - val_loss: 0.7070 - val_accuracy: 0.6985
Epoch 72/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7054 - accuracy: 0.7074 - val_loss: 0.7108 - val_accuracy: 0.6939
Epoch 73/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7059 - accuracy: 0.7023 - val_loss: 0.7050 - val_accuracy: 0.6985
Epoch 74/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7048 - accuracy: 0.6983 - val_loss: 0.7031 - val_accuracy: 0.6939
Epoch 75/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7069 - accuracy: 0.7006 - val_loss: 0.6975 - val_accuracy: 0.7045
Epoch 

...

Epoch 65/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7985 - accuracy: 0.6401 - val_loss: 0.7986 - val_accuracy: 0.6167
Epoch 66/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7991 - accuracy: 0.6276 - val_loss: 0.7965 - val_accuracy: 0.6212
Epoch 67/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7978 - accuracy: 0.6407 - val_loss: 0.7961 - val_accuracy: 0.6227
Epoch 68/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7971 - accuracy: 0.6413 - val_loss: 0.7947 - val_accuracy: 0.6379
Epoch 69/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7963 - accuracy: 0.6390 - val_loss: 0.7952 - val_accuracy: 0.6212
Epoch 70/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7960 - accuracy: 0.6384 - val_loss: 0.7958 - val_accuracy: 0.6197
Epoch 71/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7960 - accuracy: 0.6345 - val_loss: 0.7938 - val_accuracy: 0.6273
Epoch 

55/55 [==============================] - 0s 2ms/step - loss: 0.8805 - accuracy: 0.5869 - val_loss: 0.8704 - val_accuracy: 0.6015
Epoch 14/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8743 - accuracy: 0.5932 - val_loss: 0.8676 - val_accuracy: 0.6091
Epoch 15/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8697 - accuracy: 0.5909 - val_loss: 0.8569 - val_accuracy: 0.5985
Epoch 16/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8652 - accuracy: 0.5943 - val_loss: 0.8547 - val_accuracy: 0.6091
Epoch 17/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8617 - accuracy: 0.6011 - val_loss: 0.8529 - val_accuracy: 0.6197
Epoch 18/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8585 - accuracy: 0.5994 - val_loss: 0.8489 - val_accuracy: 0.6182
Epoch 19/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8552 - accuracy: 0.6062 - val_loss: 0.8459 - val_accuracy: 0.6197
Epoch 20/100
55/55 

Epoch 71/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8044 - accuracy: 0.6341 - val_loss: 0.7971 - val_accuracy: 0.6333
Epoch 72/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8038 - accuracy: 0.6273 - val_loss: 0.8010 - val_accuracy: 0.6242
Epoch 73/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8032 - accuracy: 0.6318 - val_loss: 0.8003 - val_accuracy: 0.6242
Epoch 74/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8031 - accuracy: 0.6307 - val_loss: 0.7974 - val_accuracy: 0.6333
Epoch 75/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8023 - accuracy: 0.6313 - val_loss: 0.7978 - val_accuracy: 0.6333
Epoch 76/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8018 - accuracy: 0.6341 - val_loss: 0.7948 - val_accuracy: 0.6318
Epoch 77/100
55/55 [==============================] - 0s 3ms/step - loss: 0.8015 - accuracy: 0.6313 - val_loss: 0.7953 - val_accuracy: 0.6333
Epoch 

55/55 [==============================] - 0s 1ms/step - loss: 0.6432 - accuracy: 0.7283
Model: "fnn_np"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_59 (Flatten)         (None, 217)               0         
_________________________________________________________________
dense_212 (Dense)            (None, 67)                14606     
_________________________________________________________________
dense_213 (Dense)            (None, 67)                4556      
_________________________________________________________________
dense_214 (Dense)            (None, 67)                4556      
_________________________________________________________________
dense_215 (Dense)            (None, 67)                4556      
_________________________________________________________________
dense_216 (Dense)            (None, 3)                 204       
Total params: 28,478
Trainable params: 

55/55 [==============================] - 0s 3ms/step - loss: 0.6870 - accuracy: 0.6938 - val_loss: 0.7139 - val_accuracy: 0.6773
Epoch 16/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6831 - accuracy: 0.6955 - val_loss: 0.7037 - val_accuracy: 0.7136
Epoch 17/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7001 - accuracy: 0.6903 - val_loss: 0.7500 - val_accuracy: 0.6742
Epoch 18/100
55/55 [==============================] - 0s 3ms/step - loss: 0.6868 - accuracy: 0.7000 - val_loss: 0.6857 - val_accuracy: 0.7030
Epoch 19/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6838 - accuracy: 0.6983 - val_loss: 0.6733 - val_accuracy: 0.7182
Epoch 20/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6707 - accuracy: 0.7085 - val_loss: 0.6652 - val_accuracy: 0.7121
Epoch 21/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6709 - accuracy: 0.7102 - val_loss: 0.6721 - val_accuracy: 0.7121
Epoch 22/100
55/55 

In [30]:
search_2.best_score_

0.7101174563158187

In [31]:
search_2.best_params_

{'clf__activation': 'elu',
 'clf__kernel_initializer': 'glorot_uniform',
 'clf__n_hidden': 4,
 'clf__n_neurons': 67,
 'clf__optimizer': 'rmsprop'}

In [32]:
search_2.best_estimator_

Pipeline(memory=None,
     steps=[('preprocess', Pipeline(memory=None,
     steps=[('addcolumnnames', AddColumnNames(columns=array(['lpc_1', 'lpc_2', ..., 'var_spec_rolloff', 'var_zcr'], dtype=object))), ('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('predictors_all', Pipeline(memory=None,
     steps=[('col..., ('clf', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fd190458828>)])

### CNN With Standardized Input

In [33]:
param_dist = {
    'clf__n_hidden': [1, 2, 3],
    'clf__activation': ['relu', 'elu', 'selu'],
    'clf__optimizer': ['rmsprop','adam','adagrad'],
    'clf__kernel_initializer':['glorot_uniform','normal','uniform'],
    'clf__n_neurons': numpy.arange(1,100),
    'clf__filters': [16, 32, 64],
#     'clf__kernel_size': [3,4,5],
    'clf__dropout':[.1, .15, .2, .25]
}

In [34]:
# wrap the function with keras wrapper
cnn_1 = KerasClassifier(build_fn=set_shape_create_cnn_model('cnn', ncols))

In [35]:
pipeline_sp = Pipeline([
    ('preprocess', preprocess_sp),
    ('clf', cnn_1)
])

In [36]:
# if you're not using a GPU, you can set n_jobs to something other than 1
rscv = RandomizedSearchCV(pipeline_sp, param_dist, cv=3, n_jobs=1)

In [37]:
# %%timeit
# fit model
search_3 = rscv.fit(X_train, y_train, clf__batch_size=32,
                    clf__validation_data=(X_test_sp, y_test),
                    clf__epochs=100, clf__callbacks=[early_stopping_cb])

Model: "cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 215, 32)           128       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 107, 32)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 105, 64)           6208      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 52, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 50, 96)            18528     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 25, 96)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 23, 128)           36992   

55/55 [==============================] - 1s 13ms/step - loss: 0.5313 - accuracy: 0.7879 - val_loss: 0.6807 - val_accuracy: 0.7030
Epoch 42/100
55/55 [==============================] - 1s 14ms/step - loss: 0.5280 - accuracy: 0.7868 - val_loss: 0.6811 - val_accuracy: 0.7045
Epoch 43/100
55/55 [==============================] - 1s 14ms/step - loss: 0.5290 - accuracy: 0.7800 - val_loss: 0.6875 - val_accuracy: 0.7121
Epoch 44/100
55/55 [==============================] - 0s 5ms/step - loss: 0.5439 - accuracy: 0.7919
Model: "cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 215, 32)           128       
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 107, 32)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 105, 64)           6208      
_________

Epoch 36/100
55/55 [==============================] - 1s 14ms/step - loss: 0.5455 - accuracy: 0.7783 - val_loss: 0.6684 - val_accuracy: 0.7273
Epoch 37/100
55/55 [==============================] - 1s 14ms/step - loss: 0.5369 - accuracy: 0.7891 - val_loss: 0.6680 - val_accuracy: 0.7182
Epoch 38/100
55/55 [==============================] - 1s 14ms/step - loss: 0.5333 - accuracy: 0.7828 - val_loss: 0.6704 - val_accuracy: 0.7182
Epoch 39/100
55/55 [==============================] - 1s 14ms/step - loss: 0.5255 - accuracy: 0.7857 - val_loss: 0.6733 - val_accuracy: 0.7167
Epoch 40/100
55/55 [==============================] - 1s 14ms/step - loss: 0.5209 - accuracy: 0.7891 - val_loss: 0.6694 - val_accuracy: 0.7182
Epoch 41/100
55/55 [==============================] - 1s 14ms/step - loss: 0.5211 - accuracy: 0.7845 - val_loss: 0.6700 - val_accuracy: 0.7167
Epoch 42/100
55/55 [==============================] - 1s 14ms/step - loss: 0.5140 - accuracy: 0.7948 - val_loss: 0.6703 - val_accuracy: 0.7167

Epoch 28/100
55/55 [==============================] - 1s 13ms/step - loss: 0.5955 - accuracy: 0.7528 - val_loss: 0.6691 - val_accuracy: 0.7197
Epoch 29/100
55/55 [==============================] - 1s 13ms/step - loss: 0.6048 - accuracy: 0.7534 - val_loss: 0.6706 - val_accuracy: 0.7076
Epoch 30/100
55/55 [==============================] - 1s 13ms/step - loss: 0.5994 - accuracy: 0.7557 - val_loss: 0.6770 - val_accuracy: 0.7091
Epoch 31/100
55/55 [==============================] - 1s 13ms/step - loss: 0.5945 - accuracy: 0.7540 - val_loss: 0.6700 - val_accuracy: 0.7167
Epoch 32/100
55/55 [==============================] - 1s 13ms/step - loss: 0.5862 - accuracy: 0.7619 - val_loss: 0.6672 - val_accuracy: 0.7121
Epoch 33/100
55/55 [==============================] - 1s 13ms/step - loss: 0.5749 - accuracy: 0.7642 - val_loss: 0.6633 - val_accuracy: 0.7121
Epoch 34/100
55/55 [==============================] - 1s 13ms/step - loss: 0.5736 - accuracy: 0.7818 - val_loss: 0.6640 - val_accuracy: 0.7121

Epoch 1/100
55/55 [==============================] - 1s 16ms/step - loss: 0.8903 - accuracy: 0.5867 - val_loss: 0.7742 - val_accuracy: 0.6606
Epoch 2/100
55/55 [==============================] - 1s 14ms/step - loss: 0.7332 - accuracy: 0.6885 - val_loss: 0.6953 - val_accuracy: 0.7000
Epoch 3/100
55/55 [==============================] - 1s 14ms/step - loss: 0.6180 - accuracy: 0.7425 - val_loss: 0.7460 - val_accuracy: 0.6667
Epoch 4/100
55/55 [==============================] - 1s 14ms/step - loss: 0.5148 - accuracy: 0.7993 - val_loss: 0.8631 - val_accuracy: 0.6652
Epoch 5/100
55/55 [==============================] - 1s 14ms/step - loss: 0.4235 - accuracy: 0.8277 - val_loss: 0.9167 - val_accuracy: 0.6758
Epoch 6/100
55/55 [==============================] - 1s 14ms/step - loss: 0.3382 - accuracy: 0.8732 - val_loss: 0.8971 - val_accuracy: 0.7061
Epoch 7/100
55/55 [==============================] - 1s 14ms/step - loss: 0.2540 - accuracy: 0.9045 - val_loss: 1.0307 - val_accuracy: 0.6833
Epoch 

...

55/55 [==============================] - 1s 11ms/step - loss: 0.7173 - accuracy: 0.6958 - val_loss: 0.7181 - val_accuracy: 0.6985
Epoch 4/100
55/55 [==============================] - 1s 11ms/step - loss: 0.6859 - accuracy: 0.7203 - val_loss: 0.7084 - val_accuracy: 0.7182
Epoch 5/100
55/55 [==============================] - 1s 11ms/step - loss: 0.6722 - accuracy: 0.7175 - val_loss: 0.6847 - val_accuracy: 0.7152
Epoch 6/100
55/55 [==============================] - 1s 11ms/step - loss: 0.6564 - accuracy: 0.7379 - val_loss: 0.6912 - val_accuracy: 0.7197
Epoch 7/100
55/55 [==============================] - 1s 11ms/step - loss: 0.6406 - accuracy: 0.7391 - val_loss: 0.6789 - val_accuracy: 0.7212
Epoch 8/100
55/55 [==============================] - 1s 11ms/step - loss: 0.6306 - accuracy: 0.7328 - val_loss: 0.6737 - val_accuracy: 0.7242
Epoch 9/100
55/55 [==============================] - 1s 11ms/step - loss: 0.6277 - accuracy: 0.7374 - val_loss: 0.6724 - val_accuracy: 0.7273
Epoch 10/100
55/55

55/55 [==============================] - 1s 13ms/step - loss: 0.8928 - accuracy: 0.5858 - val_loss: 0.7765 - val_accuracy: 0.6591
Epoch 2/100
55/55 [==============================] - 1s 11ms/step - loss: 0.7816 - accuracy: 0.6585 - val_loss: 0.7427 - val_accuracy: 0.6667
Epoch 3/100
55/55 [==============================] - 1s 11ms/step - loss: 0.7345 - accuracy: 0.6818 - val_loss: 0.7042 - val_accuracy: 0.7106
Epoch 4/100
55/55 [==============================] - 1s 11ms/step - loss: 0.7144 - accuracy: 0.6955 - val_loss: 0.6919 - val_accuracy: 0.7197
Epoch 5/100
55/55 [==============================] - 1s 11ms/step - loss: 0.6916 - accuracy: 0.7119 - val_loss: 0.6782 - val_accuracy: 0.7273
Epoch 6/100
55/55 [==============================] - 1s 11ms/step - loss: 0.6723 - accuracy: 0.7136 - val_loss: 0.6724 - val_accuracy: 0.7333
Epoch 7/100
55/55 [==============================] - 1s 11ms/step - loss: 0.6612 - accuracy: 0.7312 - val_loss: 0.6711 - val_accuracy: 0.7258
Epoch 8/100
55/55 

55/55 [==============================] - 1s 11ms/step - loss: 0.4212 - accuracy: 0.8420 - val_loss: 0.6165 - val_accuracy: 0.7439
Epoch 59/100
55/55 [==============================] - 1s 11ms/step - loss: 0.4157 - accuracy: 0.8386 - val_loss: 0.6157 - val_accuracy: 0.7424
Epoch 60/100
55/55 [==============================] - 1s 11ms/step - loss: 0.4149 - accuracy: 0.8392 - val_loss: 0.6136 - val_accuracy: 0.7470
Epoch 61/100
55/55 [==============================] - 1s 11ms/step - loss: 0.4104 - accuracy: 0.8415 - val_loss: 0.6191 - val_accuracy: 0.7439
Epoch 62/100
55/55 [==============================] - 1s 11ms/step - loss: 0.4071 - accuracy: 0.8403 - val_loss: 0.6156 - val_accuracy: 0.7455
Epoch 63/100
55/55 [==============================] - 1s 11ms/step - loss: 0.4055 - accuracy: 0.8506 - val_loss: 0.6200 - val_accuracy: 0.7424
Epoch 64/100
55/55 [==============================] - 1s 11ms/step - loss: 0.4014 - accuracy: 0.8500 - val_loss: 0.6194 - val_accuracy: 0.7439
Epoch 65/100

Epoch 1/100
55/55 [==============================] - 1s 12ms/step - loss: 0.9226 - accuracy: 0.5554 - val_loss: 0.8495 - val_accuracy: 0.6212
Epoch 2/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7661 - accuracy: 0.6572 - val_loss: 0.7646 - val_accuracy: 0.6379
Epoch 3/100
55/55 [==============================] - 1s 10ms/step - loss: 0.6679 - accuracy: 0.7044 - val_loss: 0.7098 - val_accuracy: 0.7015
Epoch 4/100
55/55 [==============================] - 1s 10ms/step - loss: 0.5575 - accuracy: 0.7726 - val_loss: 0.7399 - val_accuracy: 0.7015
Epoch 5/100
55/55 [==============================] - 1s 10ms/step - loss: 0.4405 - accuracy: 0.8272 - val_loss: 0.7613 - val_accuracy: 0.7121
Epoch 6/100
55/55 [==============================] - 1s 10ms/step - loss: 0.3556 - accuracy: 0.8647 - val_loss: 0.9105 - val_accuracy: 0.7106
Epoch 7/100
55/55 [==============================] - 1s 10ms/step - loss: 0.2829 - accuracy: 0.8852 - val_loss: 0.9888 - val_accuracy: 0.6894
Epoch 

Epoch 3/100
83/83 [==============================] - 1s 11ms/step - loss: 0.7359 - accuracy: 0.6874 - val_loss: 0.7113 - val_accuracy: 0.6924
Epoch 4/100
83/83 [==============================] - 1s 11ms/step - loss: 0.7081 - accuracy: 0.6938 - val_loss: 0.7061 - val_accuracy: 0.7061
Epoch 5/100
83/83 [==============================] - 1s 11ms/step - loss: 0.6922 - accuracy: 0.7097 - val_loss: 0.6875 - val_accuracy: 0.7030
Epoch 6/100
83/83 [==============================] - 1s 10ms/step - loss: 0.6727 - accuracy: 0.7215 - val_loss: 0.6766 - val_accuracy: 0.7227
Epoch 7/100
83/83 [==============================] - 1s 11ms/step - loss: 0.6671 - accuracy: 0.7196 - val_loss: 0.6664 - val_accuracy: 0.7288
Epoch 8/100
83/83 [==============================] - 1s 11ms/step - loss: 0.6480 - accuracy: 0.7302 - val_loss: 0.6607 - val_accuracy: 0.7258
Epoch 9/100
83/83 [==============================] - 1s 10ms/step - loss: 0.6443 - accuracy: 0.7351 - val_loss: 0.6535 - val_accuracy: 0.7273
Epoch 

In [38]:
search_3.best_score_

0.7184539477271354

In [39]:
search_3.best_params_

{'clf__activation': 'relu',
 'clf__dropout': 0.15,
 'clf__filters': 64,
 'clf__kernel_initializer': 'glorot_uniform',
 'clf__n_hidden': 1,
 'clf__n_neurons': 77,
 'clf__optimizer': 'adagrad'}

In [40]:
search_3.best_estimator_

Pipeline(memory=None,
     steps=[('preprocess', Pipeline(memory=None,
     steps=[('addcolumnnames', AddColumnNames(columns=array(['lpc_1', 'lpc_2', ..., 'var_spec_rolloff', 'var_zcr'], dtype=object))), ('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('predictors_with_outliers', Pipeline(memory=None,
     st..., ('clf', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fd1c018cc50>)])

### CNN With Normalized Input

In [41]:
# wrap the function with keras wrapper
cnn_2 = KerasClassifier(build_fn=set_shape_create_cnn_model('cnn_np', ncols))

In [42]:
pipeline_np = Pipeline([
    ('preprocess', preprocess_np),
    ('clf', cnn_1)
])

In [43]:
# if you're not using a GPU, you can set n_jobs to something other than 1
rscv = RandomizedSearchCV(pipeline_np, param_dist, cv=3, n_jobs=1)

In [44]:
# %%timeit
# fit model
search_4 = rscv.fit(X_train, y_train, clf__batch_size=32,
                    clf__validation_data=(X_test_np, y_test),
                    clf__epochs=100, clf__callbacks=[early_stopping_cb])

Model: "cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_89 (Conv1D)           (None, 215, 32)           128       
_________________________________________________________________
max_pooling1d_89 (MaxPooling (None, 107, 32)           0         
_________________________________________________________________
conv1d_90 (Conv1D)           (None, 105, 64)           6208      
_________________________________________________________________
max_pooling1d_90 (MaxPooling (None, 52, 64)            0         
_________________________________________________________________
conv1d_91 (Conv1D)           (None, 50, 96)            18528     
_________________________________________________________________
max_pooling1d_91 (MaxPooling (None, 25, 96)            0         
_________________________________________________________________
conv1d_92 (Conv1D)           (None, 23, 128)           36992   

55/55 [==============================] - 1s 14ms/step - loss: 0.6841 - accuracy: 0.7112 - val_loss: 0.7110 - val_accuracy: 0.6894
Epoch 42/100
55/55 [==============================] - 1s 13ms/step - loss: 0.6794 - accuracy: 0.7072 - val_loss: 0.7039 - val_accuracy: 0.6939
Epoch 43/100
55/55 [==============================] - 1s 14ms/step - loss: 0.6774 - accuracy: 0.7112 - val_loss: 0.6974 - val_accuracy: 0.6970
Epoch 44/100
55/55 [==============================] - 1s 14ms/step - loss: 0.6769 - accuracy: 0.7140 - val_loss: 0.6953 - val_accuracy: 0.6970
Epoch 45/100
55/55 [==============================] - 1s 13ms/step - loss: 0.6757 - accuracy: 0.7169 - val_loss: 0.6994 - val_accuracy: 0.7000
Epoch 46/100
55/55 [==============================] - 1s 14ms/step - loss: 0.6787 - accuracy: 0.7129 - val_loss: 0.6971 - val_accuracy: 0.7045
Epoch 47/100
55/55 [==============================] - 1s 13ms/step - loss: 0.6721 - accuracy: 0.7226 - val_loss: 0.7125 - val_accuracy: 0.6909
Epoch 48/100

55/55 [==============================] - 1s 14ms/step - loss: 0.6171 - accuracy: 0.7436 - val_loss: 0.6720 - val_accuracy: 0.7030
Epoch 99/100
55/55 [==============================] - 1s 14ms/step - loss: 0.6029 - accuracy: 0.7482 - val_loss: 0.6770 - val_accuracy: 0.7061
Epoch 100/100
55/55 [==============================] - 0s 5ms/step - loss: 0.5977 - accuracy: 0.7544
Model: "cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_93 (Conv1D)           (None, 215, 32)           128       
_________________________________________________________________
max_pooling1d_93 (MaxPooling (None, 107, 32)           0         
_________________________________________________________________
conv1d_94 (Conv1D)           (None, 105, 64)           6208      
_________________________________________________________________
max_pooling1d_94 (MaxPooling (None, 52, 64)            0         
___________________

Epoch 37/100
55/55 [==============================] - 1s 14ms/step - loss: 0.6942 - accuracy: 0.6981 - val_loss: 0.7098 - val_accuracy: 0.7000
Epoch 38/100
55/55 [==============================] - 1s 13ms/step - loss: 0.6919 - accuracy: 0.6993 - val_loss: 0.7146 - val_accuracy: 0.6985
Epoch 39/100
55/55 [==============================] - 1s 14ms/step - loss: 0.6909 - accuracy: 0.7049 - val_loss: 0.7185 - val_accuracy: 0.6894
Epoch 40/100
55/55 [==============================] - 1s 13ms/step - loss: 0.6890 - accuracy: 0.6998 - val_loss: 0.7095 - val_accuracy: 0.6939
Epoch 41/100
55/55 [==============================] - 1s 13ms/step - loss: 0.6834 - accuracy: 0.7032 - val_loss: 0.7105 - val_accuracy: 0.6955
Epoch 42/100
55/55 [==============================] - 1s 14ms/step - loss: 0.6874 - accuracy: 0.7049 - val_loss: 0.7075 - val_accuracy: 0.6955
Epoch 43/100
55/55 [==============================] - 1s 13ms/step - loss: 0.6782 - accuracy: 0.7072 - val_loss: 0.7136 - val_accuracy: 0.6985

55/55 [==============================] - 1s 14ms/step - loss: 0.6077 - accuracy: 0.7464 - val_loss: 0.6574 - val_accuracy: 0.7242
Epoch 95/100
55/55 [==============================] - 1s 14ms/step - loss: 0.6063 - accuracy: 0.7396 - val_loss: 0.6581 - val_accuracy: 0.7258
Epoch 96/100
55/55 [==============================] - 1s 13ms/step - loss: 0.6050 - accuracy: 0.7510 - val_loss: 0.6630 - val_accuracy: 0.7136
Epoch 97/100
55/55 [==============================] - 1s 14ms/step - loss: 0.6022 - accuracy: 0.7527 - val_loss: 0.6704 - val_accuracy: 0.7106
Epoch 98/100
55/55 [==============================] - 1s 14ms/step - loss: 0.6066 - accuracy: 0.7595 - val_loss: 0.6672 - val_accuracy: 0.7152
Epoch 99/100
55/55 [==============================] - 1s 14ms/step - loss: 0.5980 - accuracy: 0.7521 - val_loss: 0.6647 - val_accuracy: 0.7197
Epoch 100/100
55/55 [==============================] - 0s 5ms/step - loss: 0.5873 - accuracy: 0.7538
Model: "cnn"
_________________________________________

...

55/55 [==============================] - 1s 11ms/step - loss: 0.7132 - accuracy: 0.6998 - val_loss: 0.6994 - val_accuracy: 0.7030
Epoch 62/100
55/55 [==============================] - 1s 11ms/step - loss: 0.7090 - accuracy: 0.6981 - val_loss: 0.6989 - val_accuracy: 0.7106
Epoch 63/100
55/55 [==============================] - 1s 11ms/step - loss: 0.7116 - accuracy: 0.6953 - val_loss: 0.6992 - val_accuracy: 0.7076
Epoch 64/100
55/55 [==============================] - 1s 11ms/step - loss: 0.7047 - accuracy: 0.6976 - val_loss: 0.7045 - val_accuracy: 0.6955
Epoch 65/100
55/55 [==============================] - 1s 12ms/step - loss: 0.7016 - accuracy: 0.6993 - val_loss: 0.6975 - val_accuracy: 0.7061
Epoch 66/100
55/55 [==============================] - 1s 13ms/step - loss: 0.7065 - accuracy: 0.6970 - val_loss: 0.6972 - val_accuracy: 0.7106
Epoch 67/100
55/55 [==============================] - 1s 13ms/step - loss: 0.7039 - accuracy: 0.6924 - val_loss: 0.6971 - val_accuracy: 0.7121
Epoch 68/100

55/55 [==============================] - 1s 11ms/step - loss: 0.8578 - accuracy: 0.6131 - val_loss: 0.8234 - val_accuracy: 0.6500
Epoch 8/100
55/55 [==============================] - 1s 11ms/step - loss: 0.8390 - accuracy: 0.6182 - val_loss: 0.8264 - val_accuracy: 0.6364
Epoch 9/100
55/55 [==============================] - 1s 11ms/step - loss: 0.8320 - accuracy: 0.6182 - val_loss: 0.8088 - val_accuracy: 0.6455
Epoch 10/100
55/55 [==============================] - 1s 11ms/step - loss: 0.8311 - accuracy: 0.6187 - val_loss: 0.8042 - val_accuracy: 0.6500
Epoch 11/100
55/55 [==============================] - 1s 11ms/step - loss: 0.8201 - accuracy: 0.6392 - val_loss: 0.7929 - val_accuracy: 0.6636
Epoch 12/100
55/55 [==============================] - 1s 11ms/step - loss: 0.8110 - accuracy: 0.6295 - val_loss: 0.7881 - val_accuracy: 0.6576
Epoch 13/100
55/55 [==============================] - 1s 11ms/step - loss: 0.8114 - accuracy: 0.6381 - val_loss: 0.7892 - val_accuracy: 0.6515
Epoch 14/100
5

55/55 [==============================] - 1s 11ms/step - loss: 0.7290 - accuracy: 0.6881 - val_loss: 0.7034 - val_accuracy: 0.6924
Epoch 65/100
55/55 [==============================] - 1s 11ms/step - loss: 0.7247 - accuracy: 0.6733 - val_loss: 0.7037 - val_accuracy: 0.6970
Epoch 66/100
55/55 [==============================] - 1s 11ms/step - loss: 0.7220 - accuracy: 0.6920 - val_loss: 0.7024 - val_accuracy: 0.6924
Epoch 67/100
55/55 [==============================] - 1s 11ms/step - loss: 0.7218 - accuracy: 0.6864 - val_loss: 0.7014 - val_accuracy: 0.6970
Epoch 68/100
55/55 [==============================] - 1s 11ms/step - loss: 0.7200 - accuracy: 0.6966 - val_loss: 0.7012 - val_accuracy: 0.6970
Epoch 69/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7205 - accuracy: 0.6744 - val_loss: 0.6986 - val_accuracy: 0.6955
Epoch 70/100
55/55 [==============================] - 1s 11ms/step - loss: 0.7189 - accuracy: 0.6824 - val_loss: 0.6979 - val_accuracy: 0.7000
Epoch 71/100

Epoch 3/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7304 - accuracy: 0.6731 - val_loss: 0.6868 - val_accuracy: 0.7076
Epoch 4/100
55/55 [==============================] - 1s 10ms/step - loss: 0.6637 - accuracy: 0.7152 - val_loss: 0.7263 - val_accuracy: 0.6848
Epoch 5/100
55/55 [==============================] - 1s 10ms/step - loss: 0.6357 - accuracy: 0.7294 - val_loss: 0.7235 - val_accuracy: 0.6606
Epoch 6/100
55/55 [==============================] - 1s 10ms/step - loss: 0.6225 - accuracy: 0.7322 - val_loss: 0.6784 - val_accuracy: 0.7106
Epoch 7/100
55/55 [==============================] - 1s 10ms/step - loss: 0.5634 - accuracy: 0.7737 - val_loss: 0.7736 - val_accuracy: 0.6848
Epoch 8/100
55/55 [==============================] - 1s 10ms/step - loss: 0.5385 - accuracy: 0.7845 - val_loss: 0.6811 - val_accuracy: 0.7136
Epoch 9/100
55/55 [==============================] - 1s 11ms/step - loss: 0.4843 - accuracy: 0.7965 - val_loss: 0.8358 - val_accuracy: 0.6667
Epoch 

Epoch 1/100
55/55 [==============================] - 1s 13ms/step - loss: 1.0071 - accuracy: 0.4699 - val_loss: 0.8464 - val_accuracy: 0.5833
Epoch 2/100
55/55 [==============================] - 1s 10ms/step - loss: 0.8034 - accuracy: 0.6210 - val_loss: 0.7351 - val_accuracy: 0.6727
Epoch 3/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7203 - accuracy: 0.6773 - val_loss: 0.7125 - val_accuracy: 0.6894
Epoch 4/100
55/55 [==============================] - 1s 10ms/step - loss: 0.6862 - accuracy: 0.6926 - val_loss: 0.6775 - val_accuracy: 0.7015
Epoch 5/100
55/55 [==============================] - 1s 10ms/step - loss: 0.6549 - accuracy: 0.7080 - val_loss: 0.6763 - val_accuracy: 0.7015
Epoch 6/100
55/55 [==============================] - 1s 10ms/step - loss: 0.6133 - accuracy: 0.7483 - val_loss: 0.6416 - val_accuracy: 0.7212
Epoch 7/100
55/55 [==============================] - 1s 10ms/step - loss: 0.5523 - accuracy: 0.7585 - val_loss: 0.6594 - val_accuracy: 0.7136
Epoch 

In [45]:
search_4.best_score_

0.7226221892995426

In [46]:
search_4.best_params_

{'clf__activation': 'selu',
 'clf__dropout': 0.1,
 'clf__filters': 16,
 'clf__kernel_initializer': 'uniform',
 'clf__n_hidden': 3,
 'clf__n_neurons': 39,
 'clf__optimizer': 'adam'}

In [47]:
search_4.best_estimator_

Pipeline(memory=None,
     steps=[('preprocess', Pipeline(memory=None,
     steps=[('addcolumnnames', AddColumnNames(columns=array(['lpc_1', 'lpc_2', ..., 'var_spec_rolloff', 'var_zcr'], dtype=object))), ('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('predictors_all', Pipeline(memory=None,
     steps=[('col..., ('clf', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fd174bf5ba8>)])

In [48]:
# np.random.seed(23456)

# # Initiating an empty neural network
# cnn_model_2 = Sequential(name='cnn_2')

# # Adding convolutional layer
# cnn_model_2.add(layers.Conv1D(filters=16,
#                      kernel_size=(3),
#                      activation='relu',
#                      input_shape=(ncols,1)))

# # Adding max pooling layer
# cnn_model_2.add(layers.MaxPooling1D(pool_size=(2)))

# # Adding convolutional layer
# cnn_model_2.add(layers.Conv1D(filters=32,
#                      kernel_size=(3),
#                      activation='relu'))

# # Adding max pooling layer
# cnn_model_2.add(layers.MaxPooling1D(pool_size=(2)))

# # Adding a flattened layer to input our image data
# cnn_model_2.add(layers.Flatten())

# # Adding a dense layer with 64 neurons
# cnn_model_2.add(layers.Dense(64, activation='relu'))

# # Adding a dropout layer for regularization
# cnn_model_2.add(layers.Dropout(0.25))

# # Adding an output layer
# cnn_model_2.add(layers.Dense(4, activation='softmax'))

# # Compiling our neural network
# cnn_model_2.compile(loss='categorical_crossentropy',
#                   optimizer='adam',
#                   metrics=['accuracy'])

# # Fitting our neural network
# history = cnn_model_2.fit(X_train_np,
#                         y_train, 
#                         batch_size=16,
#                         validation_data=(X_test_np, y_test),
#                         epochs=15)